In [ ]:
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import class_weight

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
print('Tensorflow version:', tf.__version__)

#### Exploratory analysis of small subset of images

In [ ]:
with open('test_subjects','rb') as f:
               images = pickle.load(f)

In [ ]:
print('Number of records:', len(images))

In [ ]:
print('Record names:', images.keys())

In [ ]:
print('First record keys:', images['OAS30001'].keys())

In [ ]:
print('Shape of MRI:', images['OAS30001']['image'].shape)

In [ ]:
print('Value of a center voxel:', images['OAS30001']['image'][60][80][60])

In [ ]:
# CN label
print('Center (60th) 160x120 image of first CN MRI:')
plt.imshow(images['OAS30001']['image'][60], interpolation='nearest')
plt.show()

In [ ]:
# CN label
print('All 160x120 images of first CN MRI:') 
fig, axes = plt.subplots(12,10, figsize=(20,20))
for i,ax in enumerate(axes.flat):
    ax.imshow(images['OAS30001']['image'][i])

In [ ]:
# AD label
print('Center (60th) 160x120 image of first AD MRI:')
plt.imshow(images['OAS30024']['image'][60], interpolation='nearest')
plt.show()

In [ ]:
# AD label
print('All 160x120 images of first AD MRI:')
fig, axes = plt.subplots(12,10, figsize=(20,20))
for i,ax in enumerate(axes.flat):
    ax.imshow(images['OAS30024']['image'][i])

##### --- Start adaptation from Arezoo's notebook ---

#### Subject IDs

In [ ]:
im_name_list = []

for x1 in images:
    im_name_list.append(x1)
im_name_list

# im_name_list : list of image names

#### Making training and test sets

In [ ]:
pixels = np.zeros((1, 120, 160, 120))
for name in im_name_list:
    pixels = np.append(pixels ,np.reshape(images[name]['image'], (1, 120, 160, 120)), axis=0)

# print(pixels.shape)
# pixels : initial zeroes row and 4D representation of images
# input_ims : 4D representation of images

input_ims = pixels[1:12, :, :, :] 
input_ims.shape

In [ ]:
x_train = input_ims[0:9, :, :, :] # x_train : first 9 records
x_train.shape

In [ ]:
x_test = input_ims[9:11, :, :, :] # x_test : last 2 records
x_test.shape

In [ ]:
x_test = np.reshape(x_test, (2, 120, 160, 120, 1))

#### Making target values

In [ ]:
y_train = [images[key]['group'] for key in images.keys()]
y_test = y_train[-2:] # y_test : last 2 records
y_test

In [ ]:
y_train = y_train[:9] # y_train : first 9 records
y_train

In [ ]:
def convert_numeric(x):
    if x == "AD":
        return 1
    elif x == "CN":
        return 0
    else:
        return x 


In [ ]:
y_train = list(map(convert_numeric, y_train))
y_train

In [ ]:
y_test = list(map(convert_numeric, y_test))
y_test

In [ ]:
len(input_ims)

In [ ]:
#looking into first image
first_im = images['OAS30001']
first = first_im['image']
first.shape

In [ ]:
from sklearn.utils import class_weight

balancing_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

balancing_weights = dict(enumerate(balancing_weights))
balancing_weights

In [ ]:
first_modified = np.reshape(first, (120, 160, 120, 1))

In [ ]:
first_modified.shape

In [ ]:
x_train = np.reshape(x_train, (9, 120, 160, 120, 1))
x_test = np.reshape(x_test, (2, 120, 160, 120, 1)) 

In [ ]:
y_train = np.reshape(y_train, (9, 1))
y_test = np.reshape(y_test, (2, 1)) 

#### Building Encoder

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv3D, MaxPool3D, Dropout, \
    Flatten, Conv3DTranspose, UpSampling3D, Reshape
from tensorflow.keras.layers import Input


In [ ]:
input_img = Input(shape=(120, 160, 120 , 1), name='Input')

print('\ninput shape:', input_img.shape)

x = Conv3D(10, (3,3,3), activation='relu', padding='same', name='Convolution_1')(input_img)
x = MaxPool3D((2,2,2), name='MaxPooling_1')(x)
x = Dropout(0.8, name='DropOut_1')(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same', name='Convolution_2')(x)
x = MaxPool3D((2,2,2), name='MaxPooling_2')(x)
x = Dropout(0.8, name='DropOut_2')(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same', name='Convolution_3')(x)
x = MaxPool3D((2,2,2), name='MaxPooling_3')(x)
x = Dropout(0.8, name='DropOut_3')(x)

conv_shape = K.int_shape(x)

print('shape after convolutions:', conv_shape)

# x = Flatten(name='Flatten')(x)

# x = Dense(32, activation='selu', name='SELU_1')(x)
# x = Dense(16, activation='selu', name='SELU_2')(x)

# encoded = Dense(2, name='Encoded')(x)

# encoded_shape = K.int_shape(encoded)

# print('final encoded shape:', encoded_shape, '\n')

encoded = x
encoded_shape = K.int_shape(encoded)


encoder = Model(input_img, encoded, name='CAE_encoder')
encoder.summary()


In [ ]:
# model only: post-encoder

# x = Flatten(name='Flatten')(x)

# x = Dense(32, activation='selu', name='SELU_1')(x)
# x = Dense(16, activation='selu', name='SELU_2')(x)

# encoded = Dense(2, name='Encoded')(x)

# encoded_shape = K.int_shape(encoded)

# print('final encoded shape:', encoded_shape, '\n')

# encoder = Model(input_img, encoded, name='CAE_encoder')
# encoder.summary()


##### CAE Encoder layers

Input

    120 x 160 x 120 x 1

Convolutional layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    120 x 160 x 120 x 10

Maxpooling downsample (2x2x2 kernel)

    60 x 80 x 60 x 10

Dropout (80% of nodes set to 0)

    60 x 80 x 60 x 10

Convolutional layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    60 x 80 x 60 x 10

Maxpooling downsample (2x2x2 kernel)

    30 x 40 x 30 x 10

Dropout (80% of nodes set to 0)

    30 x 40 x 30 x 10

Convolutional layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    30 x 40 x 30 x 10

Maxpooling downsample (2x2x2 kernel)

    15 x 20 x 15 x 10

Dropout (80% of nodes set to 0)

    15 x 20 x 15 x 10
    

<!-- ###### In the Classifier only:

Flatten

    45000

Dense layer (32 channels, SELU activation)

    32

Dense layer (16 channels, SELU activation)

    16

Dense layer (2 possible outputs)

    2
 -->

#### Building Decoder

In [ ]:
encoded_input = Input(shape=(encoded_shape[1],encoded_shape[2], encoded_shape[3], encoded_shape[4],), name='Encoded')

x = encoded_input

print('\ninput shape:', encoded_input.shape)

# x = Dense(16, activation='selu', name='SELU_2')(encoded_input)
# x = Dense(32, activation='selu', name='SELU_1')(x)
# x = Dense(np.prod(conv_shape[1:]), name='Product')(x)
# x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3], conv_shape[4]), name='Reshape')(x)

# print('shape after reshape:', K.int_shape(x))

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same', name='Convolution_3')(x)
x = UpSampling3D((2,2,2), name='UpSampling_3')(x)
x = Dropout(0.8, name='DropOut_3')(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same', name='Convolution_2')(x)
x = UpSampling3D((2,2,2), name='UpSampling_2')(x)
x = Dropout(0.8, name='DropOut_2')(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same', name='Convolution_1')(x)
x = UpSampling3D((2,2,2), name='UpSampling_1')(x)
x = Dropout(0.8, name='DropOut_1')(x)

# decoded = Dense(1, activation='sigmoid', name='Output')(x)
decoded = Conv3DTranspose(1, (3,3,3), activation='sigmoid', padding='same', name='Output')(x)

print('final decoded shape:', K.int_shape(decoded), '\n')

decoder = Model(encoded_input, decoded, name='CAE_decoder')
decoder.summary()


<!-- 
 ###### In the Classifier only:


Input

    2

Dense layer (16 channels, SELU activation)

    16

Dense layer (32 channels, SELU activation)

    32

Dense layer (product of encoded_shape dimensions 15x20x15x10)

    45000

Reshape to encoded_shape dimensions

    15 x 20 x 15 x 10 
    
     -->
##### CAE Decoder layers

Input

    15 x 20 x 15 x 10

Convolutional transpose layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    15 x 20 x 15 x 10

Upsampling (2x2x2 kernel)

    30 x 40 x 30 x 10

Dropout (80% of nodes set to 0)

    30x 40 x 30 x 10

Convolutional transpose layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    30 x 40 x 30 x 10

Upsampling (2x2x2 kernel)

    60 x 80 x 60 x 10

Dropout (80% of nodes set to 0)

    60 x 80 x 60 x 10

Convolutional transpose layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    60 x 80 x 60 x 10

Upsampling (2x2x2 kernel)

    120 x 160 x 120 x 10

Dropout (80% of nodes set to 0)

    120 x 160 x 120 x 10

Convolutional transpose layer (1 channel, 3x3x3 kernel, sigmoid activation, padding)

    120 x 160 x 120 x 1


#### Building Autoencoder

In [ ]:
autoencoder = Model(input_img, decoder(encoder(input_img)), name='CAE_autoencoder')
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

hist = autoencoder.fit(x_train, x_train, epochs=1, verbose=1, validation_data=(x_test, x_test)) # epochs=200

##### --- End adaption from Arezoo's notebook ---

### ICAE Inception Model

#### Building Encoder

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv3D, MaxPool3D, Dropout, Flatten, \
    Conv3DTranspose, UpSampling3D, Reshape
from tensorflow.keras.layers import Input

from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l1_l2


In [ ]:
def inception_module(input, filter_operation):
    """
    filter_operation is a list of inception operations for:
        1x1x1 kernel, 
        1x1x1 then 3x3x3 kernels, 
        1x1x1 then 3x3x3 then 3x3x3 kernels, 
        and 3x3x3 maxpooling then 1x1x1 kernels, 
            respectively.
    """
    
    # According to Nature paper (Oh et. al.), l1 and l2 values of 10e-4 performed the best 
    #   from experimentation with 0.01, 0.001, 0.0001, 0.00001 values.
    l1_value = 10e-4
    l2_value = 10e-4

    # Branch A
    branch_1x1x1 = Conv3D(filter_operation[0], kernel_size=(1, 1, 1), activation='relu', \
                          padding='same', kernel_regularizer=l1_l2(l1_value, l2_value), \
                          name='Branch_A_1x1x1')(input)

    # Branch B
    branch_3x3x3_initial = Conv3D(filter_operation[1], kernel_size=(1, 1, 1), activation='relu', \
                                  padding='same', kernel_regularizer=l1_l2(l1_value, l2_value), \
                                  name='Branch_B_1x1x1')(input)
    branch_3x3x3 = Conv3D(filter_operation[1], kernel_size=(3, 3, 3), activation='relu', \
                          padding='same', kernel_regularizer=l1_l2(l1_value, l2_value), \
                          name='Branch_B_3x3x3')(branch_3x3x3_initial)

    # Branch C
    branch_double_3x3x3_initial = Conv3D(filter_operation[2], kernel_size=(1, 1, 1), \
                                         activation='relu', padding='same', \
                                         kernel_regularizer=l1_l2(l1_value, l2_value), \
                                         name='Branch_C_1x1x1')(input)
    branch_double_3x3x3_middle = Conv3D(filter_operation[2], kernel_size=(3, 3, 3), \
                                        activation='relu', padding='same', \
                                        kernel_regularizer=l1_l2(l1_value, l2_value), \
                                        name='Branch_C_1st_3x3x3')(branch_double_3x3x3_initial)
    branch_double_3x3x3 = Conv3D(filter_operation[2], kernel_size=(3, 3, 3), activation='relu', \
                                 padding='same', kernel_regularizer=l1_l2(l1_value, l2_value), \
                                 name='Branch_C_2nd_3x3x3')(branch_double_3x3x3_middle)

    # Branch D
    branch_maxpool_3x3x3_initial = MaxPool3D(pool_size=(3, 3, 3), strides=(1, 1, 1), \
                                             padding='same', name='Branch_D_3x3x3_maxpool')(input)
    branch_maxpool_3x3x3 = Conv3D(filter_operation[3], kernel_size=(1, 1, 1), activation='relu', \
                                  padding='same', kernel_regularizer=l1_l2(l1_value, l2_value), \
                                  name='Branch_D_1x1x1')(branch_maxpool_3x3x3_initial)
    
    # Merge branches
    modules = [branch_1x1x1, branch_3x3x3, branch_double_3x3x3, branch_maxpool_3x3x3]
    merged_module = concatenate(modules, name='Inception_Merged') # axis=-1
    
    return merged_module
    

In [ ]:
input_img = Input(shape= (120, 160, 120 , 1), name='Input')

print('\ninput shape:', input_img.shape)

x = Conv3D(10, (3,3,3), activation='relu', padding='same', name='Convolution_1')(input_img)
x = MaxPool3D((2,2,2), name='MaxPooling_1')(x)
x = Dropout(0.8, name='DropOut_1')(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same', name='Convolution_2')(x)
x = MaxPool3D((2,2,2), name='MaxPooling_2')(x)
x = Dropout(0.8, name='DropOut_2')(x)

# filter_operations = [40, 40, 40, 40]
filter_operations = [10, 10, 10, 10]

x = inception_module(x, filter_operations)
x = MaxPool3D((2,2,2), name='Inception_MaxPooling')(x)
encoded = Dropout(0.8, name='DropOut_3')(x)

# print('shape after inception:', K.int_shape(x))

# ## for model only
# encoded = Flatten(name='Flatten')(encoded)
# encoded = Dense(1, activation='sigmoid', name='Prediction')(encoded)
# ##

encoder = Model(input_img, encoded, name='ICAE_encoder')

encoded_shape = K.int_shape(encoded)
print('final encoded shape:', encoded_shape, '\n')

encoder.summary(positions=[.35, .64, .71, 1.]) # adjusts print settings to minimize truncation


##### ICAE Encoder layers

Input

    120 x 160 x 120 x 1

Convolutional layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    120 x 160 x 120 x 10

Maxpooling downsample (2x2x2 kernel)

    60 x 80 x 60 x 10

Convolutional layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    60 x 80 x 60 x 10

Maxpooling downsample (2x2x2 kernel)

    30 x 40 x 30 x 10

Dropout (80% of nodes set to 0)

    30 x 40 x 30 x 10

Inception branched layers

    A: 30 x 40 x 30 x 10

    B: 30 x 40 x 30 x 10  ->  30 x 40 x 30 x 10

    C: 30 x 40 x 30 x 10  ->  30 x 40 x 30 x 10  ->  30 x 40 x 30 x 10

    D: 30 x 40 x 30 x 10  ->  30 x 40 x 30 x 10

Inception merged layer

    30 x 40 x 30 x 40

Maxpooling downsample (2x2x2 kernel)

    15 x 20 x 15 x 40

Dense layer (2 possible outputs, SoftMax activation)

    15 x 20 x 15 x 2


#### Building Decoder

In [ ]:
encoded_input= Input(shape=(encoded_shape[1],encoded_shape[2], \
                            encoded_shape[3], encoded_shape[4],), name='Input')

print('\ninput shape:', encoded_input.shape)

# filter_operations = [40, 40, 40, 40]
# filter_operations = [10, 10, 10, 10]
combined_dim = sum(filter_operations) # taken from encoder

# x = Dense(combined_dim, activation='selu', name='SELU')(encoded_input)

x = encoded_input

x = inception_module(x, filter_operations)

# print('shape after inception:', K.int_shape(x))

x = UpSampling3D((2,2,2), name='Inception_Upsampling')(x)
x = Dropout(0.8, name='DropOut_3')(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same', name='Convolution_2')(x)
x = UpSampling3D((2,2,2), name='Upsampling_2')(x)
x = Dropout(0.8, name='DropOut_2')(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same', name='Convolution_1')(x)
x = UpSampling3D((2,2,2), name='Upsampling_1')(x)
x = Dropout(0.8, name='DropOut_1')(x)

# decoded = Dense(1, activation='sigmoid', name='Output')(x)
decoded = Conv3DTranspose(1, (3,3,3), activation='sigmoid', padding='same', name='Output')(x)

print('final decoded shape:', K.int_shape(decoded), '\n')

decoder = Model(encoded_input, decoded, name='ICAE_decoder')
decoder.summary(positions=[.35, .64, .71, 1.])


##### ICAE Decoder layers

Input

    15 x 20 x 15 x 10

Inception branched layers

    A: 15 x 20 x 15 x 10

    B: 15 x 20 x 15 x 10  ->  15 x 20 x 15 x 10

    C: 15 x 20 x 15 x 10  ->  15 x 20 x 15 x 10  ->  15 x 20 x 15 x 10

    D: 15 x 20 x 15 x 10  ->  15 x 20 x 15 x 10

Upsampling (2x2x2 kernel)

    30 x 40 x 30 x 10

Dropout (80% of nodes set to 0)

    30x 40 x 30 x 10

Convolutional transpose layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    30 x 40 x 30 x 10

Upsampling (2x2x2 kernel)

    60 x 80 x 60 x 10

Dropout (80% of nodes set to 0)

    60 x 80 x 60 x 10

Convolutional transpose layer (10 channels, 3x3x3 kernel, ReLU activation, padding)

    60 x 80 x 60 x 10

Upsampling (2x2x2 kernel)

    120 x 160 x 120 x 10

Dropout (80% of nodes set to 0)

    120 x 160 x 120 x 10

Convolutional transpose layer (1 channel, 3x3x3 kernel, sigmoid activation, padding)

    120 x 160 x 120 x 1



#### Building Autoencoder

In [ ]:
ICAE_autoencoder = Model(input_img, decoder(encoder(input_img)), name='ICAE_autoencoder')
ICAE_autoencoder.summary()

In [ ]:
ICAE_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

ICAE_hist = ICAE_autoencoder.fit(x_train, x_train, epochs=1, verbose=1, \
                                 validation_data=(x_test, x_test)) # epochs=200